In [3]:
!pip install kafka-python

     |████████████████████████████████| 246 kB 537 kB/s eta 0:00:01


In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-1-5796b8bfe42c>:3 

In [3]:
df = (spark.read.format("com.databricks.spark.csv")
        .option("header", "true")
        .option("inferSchema","true")
        .load("../data/census_1000.csv"))
        
df_list = df.collect()
df.show()

+---+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+------+
|_c0|age|        workclass|    education|education-num|      marital-status|        occupation|  relationship|          ethnicity| gender|capital-gain|capital-loss|hours-per-week|  loan|
+---+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+------+
|  0| 39|        State-gov|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|        2174|           0|            40| <=50K|
|  1| 50| Self-emp-not-inc|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|           0|           0|            13| <=50K|
|  2| 38|          Private|      HS-grad|            9|          

In [ ]:
import time
import json
import random
import logging

from kafka import KafkaProducer
from kafka.errors import KafkaError

KAFKA_BROKER = "kafka:9092"
KAFKA_TOPIC = "default_topic"

producer = KafkaProducer(bootstrap_servers=[KAFKA_BROKER])
index = 0

while True:
    
    row_dict = df_list[index].asDict()
    
    future = producer.send(
        topic=KAFKA_TOPIC, 
        key=str(row_dict["_c0"]).encode("utf-8"),
        value=json.dumps(row_dict).encode("utf-8"))
    
    try:
        record_metadata = future.get(timeout=10)
    except KafkaError:
        # Decide what to do if produce request failed...
        logging.exception("Error")
        pass
    
    producer.flush()
    
    index += 1
    time.sleep(random.uniform(0.1,3.0))